# Earth Data Analytics Bootcamp Final Assignment

---

## Grassland Management Under Climate Change

** WRITE AN INTRODUCTION HERE **

In [7]:
# Import standard libraries
import logging              # Logging of messages
import os                   # Handles files and directories
import shutil               # Clear cashe
import warnings             # Show warning triggered by module import
from glob import glob       # Creates lists of files for batch processing

# Import third party libraries
import earthpy as et        # Earthpy library
import earthpy.earthexplorer as etee    # Plot and manipulate spatial data
import earthpy.spatial as es    # Used for spatial statistics
import geopandas as gpd     # Work with vector files
import geoviews as gv       # Visualization package
import holoviews as hv      # Visualization package
import hvplot.pandas        # Used for plotting data
import hvplot.xarray        # Used for plotting maps
import numpy as np          # Used for math operations
import pandas as pd         # Used for working with tabular data
import rioxarray as rxr     # Storage and manipulation of rasters
import rioxarray.merge as rxrm   # Merge rioxarrays
import xarray as xr         # Work with multidimensional arrays

# Set up logging so AppeearsDownloader will log in notebook
logging.basicConfig(level=logging.INFO)

# Ignore FutureWarning
# warnings.simplefilter(action='ignore')

# Get root directory and data directory paths and save as variables
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
grassland_dir = os.path.join(data_dir, 'grassland-units')
soil_dir = os.path.join(data_dir, 'polaris-soils')
elevation_dir = os.path.join(data_dir, 'srtm-elevation')
climate_dir = os.path.join(data_dir, 'maca_v2-climate')

# If directories do not exist, create
for a_dir in [grassland_dir, soil_dir, elevation_dir, climate_dir]:
    if not os.path.exists(a_dir):
        os.makedirs(a_dir)

In [8]:
# Check that data directories were created
print(f'\n Root Directory: {data_dir}', 
      f'\n Grassland Data Directory: {grassland_dir}',
      f'\n Soil Data Directory: {soil_dir}',
      f'\n Elevation Data Directory: {elevation_dir}',
      f'\n Climate Data Directory: {climate_dir}')


 Root Directory: /home/jovyan/earth-analytics/data 
 Grassland Data Directory: /home/jovyan/earth-analytics/data/grassland-units 
 Soil Data Directory: /home/jovyan/earth-analytics/data/polaris-soils 
 Elevation Data Directory: /home/jovyan/earth-analytics/data/srtm-elevation 
 Climate Data Directory: /home/jovyan/earth-analytics/data/maca_v2-climate


## USFS National Grassland Units

The [United States Forest Service administers twenty grassland units](https://www.fs.usda.gov/managing-land/national-forests-grasslands/national-grasslands) in twelve states composing over 3.8 million acres of public land. Each unit serves a variety of purposes related to ecology, biology, resources, and recreation. Native grasslands also provide a variety of [ecosystem services](https://www.fs.usda.gov/managing-land/national-forests-grasslands/national-grasslands/ecoservices) that benefit the health and wellness of humans and the natural world. These include but are not limited to: drought and flood mitigation, development of soil nutrients and fertility management, pest control, protection of natural water systems and from soil erosion, and sequestration of atmospheric carbon dioxide (CO2) among others.

### [Butte Valley National Grassland - California](https://www.fs.usda.gov/detail/klamath/about-forest/?cid=FSEPRD494406)

Butte Valley National Grassland is located in a scenic basin lowland in the southern Casade Range, northern California. It is California's only national grassland. The grassland is 18.425 acres in size and plays host to sagebrush, rabbitbrush, bitterbrush, basin wildrye, intermediate wheatgrass, a variety of arid grasses and flowers, and occassional western Juniper trees.

![Butte Valley Grassland](https://github.com/ellenalamont17/bootcamp_final_2023-ellenalamont17/IMG/ButteValleyGrassland.jpg)

*Photograph of the Butte Valley National Grassland looking toward Mount Shasta with a view of intermediate wheatgrass in the foreground. Photo Credit: [U.S. Forest Service](http://www.fs.fed.us/grasslands/lonetree/buttevalley.shtml)*


### [Crooked River National Grassland - Oregon](https://www.fs.usda.gov/recarea/ochoco/recarea/?recid=38274)

Crooked River National Grassland is located west of the central Cascade Range in Jefferson County, Oregon. It is Oregon's only national grassland. The grassland is 173,629 acres and contains two National Wild and Scenic Rivers - the Deschutes and Crooked rivers. Crested wheatgrass, bluebunch wheatgrass, and Idaho fescue are common.

![Crooked River Grassland](https://github.com/ellenalamont17/bootcamp_final_2023-ellenalamont17/IMG/CrookedRiverGrassland.jpg)

*Photograph of the Crooked River National Grassland looking from pinnacle ridge toward Three Sisters volcanoes. Photo Credit: [The Oregonian](https://www.oregonlive.com/terryrichard/2011/12/crooked_river_grassland_is_no.html)*

In [12]:
# Download, open, read, and select grassland units 

# Download the grassland unit shapefile if it does not exist
grassland_path = os.path.join(grassland_dir, 'S_USA.NationalGrassland.shp')

if not os.path.exists(grassland_path):
    grass_zip_url = ('https://data.fs.usda.gov/geodata/edw/edw_resources/'
                     'shp/S_USA.NationalGrassland.zip')
    gpd.read_file(grass_zip_url).to_file(grassland_path)
# grassland_path                    # Test if shapefile was downloaded to path

# Load the grassland bounary data into a gdf and set the index
grassland_bndry = gpd.read_file(grassland_path).set_index('GRASSLANDN')
# grassland_bndry                   # Test if shapefile was read to gdf


# Select specific grassland unit boundary rows
unit_bndry = (
    grassland_bndry
    .loc[['Butte Valley National Grassland',
           'Crooked River National Grassland']]
)
unit_bndry                          # Test if individual units were selected

,NATIONALGR,GIS_ACRES,SHAPE_AREA,SHAPE_LEN,geometry
GRASSLANDN,,,,,
Butte Valley National Grassland,295507010328,19489.170,0.008557,0.853736,"MULTIPOLYGON (((-121.99600 41.84049, -121.9960..."
Crooked River National Grassland,295519010328,173593.026,0.079499,2.815873,"POLYGON ((-120.98025 44.72935, -120.98030 44.7..."
